# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 28 2022 12:33PM,240637,10,8204202,MedStone Pharma LLC,Released
1,Jun 28 2022 12:33PM,240637,10,8204203,MedStone Pharma LLC,Released
2,Jun 28 2022 12:33PM,240637,10,8204204,MedStone Pharma LLC,Released
3,Jun 28 2022 12:33PM,240637,10,8204205,MedStone Pharma LLC,Released
4,Jun 28 2022 12:33PM,240637,10,8204206,MedStone Pharma LLC,Released
5,Jun 28 2022 12:30PM,240636,10,BPS1973,Beach Patient Assistance,Released
6,Jun 28 2022 12:30PM,240636,10,8204282,Beach Products Inc,Released
7,Jun 28 2022 12:27PM,240635,21,619264,"NBTY Global, Inc.",Released
8,Jun 28 2022 12:26PM,240634,21,619265,"NBTY Global, Inc.",Released
9,Jun 28 2022 12:24PM,240633,10,Eme-100612,Emerginnova,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
42,240633,Released,4
43,240634,Released,1
44,240635,Released,1
45,240636,Released,2
46,240637,Released,5


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
240633,NaN,4.0
240634,NaN,1.0
240635,NaN,1.0
240636,NaN,2.0
240637,NaN,5.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
240535,1.0,0.0
240550,0.0,1.0
240574,0.0,1.0
240577,0.0,1.0
240578,1.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
240535,1,0
240550,0,1
240574,0,1
240577,0,1
240578,1,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,240535,1,0
1,240550,0,1
2,240574,0,1
3,240577,0,1
4,240578,1,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,240535,1,
1,240550,,1
2,240574,,1
3,240577,,1
4,240578,1,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 28 2022 12:33PM,240637,10,MedStone Pharma LLC
5,Jun 28 2022 12:30PM,240636,10,Beach Patient Assistance
6,Jun 28 2022 12:30PM,240636,10,Beach Products Inc
7,Jun 28 2022 12:27PM,240635,21,"NBTY Global, Inc."
8,Jun 28 2022 12:26PM,240634,21,"NBTY Global, Inc."
9,Jun 28 2022 12:24PM,240633,10,Emerginnova
13,Jun 28 2022 12:23PM,240630,20,"ACI Healthcare USA, Inc."
28,Jun 28 2022 12:22PM,240632,22,"NBTY Global, Inc."
29,Jun 28 2022 12:21PM,240631,15,"Virtus Pharmaceuticals, LLC"
44,Jun 28 2022 12:20PM,240629,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 28 2022 12:33PM,240637,10,MedStone Pharma LLC,,5
1,Jun 28 2022 12:30PM,240636,10,Beach Patient Assistance,,2
2,Jun 28 2022 12:30PM,240636,10,Beach Products Inc,,2
3,Jun 28 2022 12:27PM,240635,21,"NBTY Global, Inc.",,1
4,Jun 28 2022 12:26PM,240634,21,"NBTY Global, Inc.",,1
5,Jun 28 2022 12:24PM,240633,10,Emerginnova,,4
6,Jun 28 2022 12:23PM,240630,20,"ACI Healthcare USA, Inc.",,15
7,Jun 28 2022 12:22PM,240632,22,"NBTY Global, Inc.",,1
8,Jun 28 2022 12:21PM,240631,15,"Virtus Pharmaceuticals, LLC",,15
9,Jun 28 2022 12:20PM,240629,22,"NBTY Global, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 28 2022 12:33PM,240637,10,MedStone Pharma LLC,5,
1,Jun 28 2022 12:30PM,240636,10,Beach Patient Assistance,2,
2,Jun 28 2022 12:30PM,240636,10,Beach Products Inc,2,
3,Jun 28 2022 12:27PM,240635,21,"NBTY Global, Inc.",1,
4,Jun 28 2022 12:26PM,240634,21,"NBTY Global, Inc.",1,
5,Jun 28 2022 12:24PM,240633,10,Emerginnova,4,
6,Jun 28 2022 12:23PM,240630,20,"ACI Healthcare USA, Inc.",15,
7,Jun 28 2022 12:22PM,240632,22,"NBTY Global, Inc.",1,
8,Jun 28 2022 12:21PM,240631,15,"Virtus Pharmaceuticals, LLC",15,
9,Jun 28 2022 12:20PM,240629,22,"NBTY Global, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 28 2022 12:33PM,240637,10,MedStone Pharma LLC,5,
1,Jun 28 2022 12:30PM,240636,10,Beach Patient Assistance,2,
2,Jun 28 2022 12:30PM,240636,10,Beach Products Inc,2,
3,Jun 28 2022 12:27PM,240635,21,"NBTY Global, Inc.",1,
4,Jun 28 2022 12:26PM,240634,21,"NBTY Global, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 28 2022 12:33PM,240637,10,MedStone Pharma LLC,5.0,NaN
1,Jun 28 2022 12:30PM,240636,10,Beach Patient Assistance,2.0,NaN
2,Jun 28 2022 12:30PM,240636,10,Beach Products Inc,2.0,NaN
3,Jun 28 2022 12:27PM,240635,21,"NBTY Global, Inc.",1.0,NaN
4,Jun 28 2022 12:26PM,240634,21,"NBTY Global, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 28 2022 12:33PM,240637,10,MedStone Pharma LLC,5.0,0.0
1,Jun 28 2022 12:30PM,240636,10,Beach Patient Assistance,2.0,0.0
2,Jun 28 2022 12:30PM,240636,10,Beach Products Inc,2.0,0.0
3,Jun 28 2022 12:27PM,240635,21,"NBTY Global, Inc.",1.0,0.0
4,Jun 28 2022 12:26PM,240634,21,"NBTY Global, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2406209,86.0,8.0
15,1684300,75.0,29.0
16,240594,2.0,5.0
17,240550,1.0,0.0
18,240614,1.0,0.0
19,1443598,27.0,5.0
20,1203102,49.0,12.0
21,1443582,5.0,1.0
22,721878,3.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2406209,86.0,8.0
1,15,1684300,75.0,29.0
2,16,240594,2.0,5.0
3,17,240550,1.0,0.0
4,18,240614,1.0,0.0
5,19,1443598,27.0,5.0
6,20,1203102,49.0,12.0
7,21,1443582,5.0,1.0
8,22,721878,3.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,86.0,8.0
1,15,75.0,29.0
2,16,2.0,5.0
3,17,1.0,0.0
4,18,1.0,0.0
5,19,27.0,5.0
6,20,49.0,12.0
7,21,5.0,1.0
8,22,3.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,86.0
1,15,Released,75.0
2,16,Released,2.0
3,17,Released,1.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,17,18,19,20,21,22
Status,,,,,,,,,
Executing,8.0,29.0,5.0,0.0,0.0,5.0,12.0,1.0,0.0
Released,86.0,75.0,2.0,1.0,1.0,27.0,49.0,5.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,17,18,19,20,21,22
0,Executing,8.0,29.0,5.0,0.0,0.0,5.0,12.0,1.0,0.0
1,Released,86.0,75.0,2.0,1.0,1.0,27.0,49.0,5.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,17,18,19,20,21,22
0,Executing,8.0,29.0,5.0,0.0,0.0,5.0,12.0,1.0,0.0
1,Released,86.0,75.0,2.0,1.0,1.0,27.0,49.0,5.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()